In [7]:
import numpy as np
from tkinter import *
from update_state import *

%reload_ext autoreload
%autoreload 2

## Define which cells are Empty, Pedestrian, Target
The below cell runs a GUI where you can toggle the state of cells by pressing and dragging the left mouse button.

The cell state cycles as E, P, T upon pressing the left click (or dragging over a group of cells).

* Empty(E) : White
* Pedestrian(P) : Red
* Target(T) : Yellow

Default Cell state is **E**(white)

In [8]:
class Cell():
    EMPTY_COLOR_BG = "white"
    EMPTY_COLOR_BORDER = "black"
    
    COLOR_DICT = {'empty':'white','pedestrian':'red', 'target':'yellow'}
    CELL_ENUM = {'empty':0,'pedestrian':1, 'target':2}
    CELL_STATE = ['empty','pedestrian', 'target']
    
    def __init__(self, master, x, y, size, init_cell_state=None):
        """ Constructor of the object called by Cell(...) """
        self.master = master
        self.x = x
        self.y = y
        self.size = size
        
        if init_cell_state==None:
            self.enum_val = 0
        else:
            self.enum_val = Cell.CELL_ENUM[init_cell_state]
        self.state = Cell.CELL_STATE[self.enum_val]
        
    def _switch(self):
        """ Toggle Cell State """
        self.enum_val += 1

    def draw(self, state=None):
        """ order to the cell to draw its representation on the canvas """
        if self.master != None :
            outline = Cell.EMPTY_COLOR_BORDER
            if state==None:
                self.state = Cell.CELL_STATE[self.enum_val % len(Cell.CELL_STATE)]
            else:
                self.state = state
            fill = Cell.COLOR_DICT[self.state]
            outline = Cell.EMPTY_COLOR_BORDER

            xmin = self.x * self.size
            xmax = xmin + self.size
            ymin = self.y * self.size
            ymax = ymin + self.size

            self.master.create_rectangle(xmin, ymin, xmax, ymax, fill = fill, outline = outline)

class CellGrid(Canvas):
    def __init__(self,master, rowNumber, columnNumber, cellSize, init_grid=None, *args, **kwargs):
        Canvas.__init__(self, master, width = cellSize * columnNumber , height = cellSize * rowNumber, *args, **kwargs)

        self.cellSize = cellSize
        
        self.num_rows = rowNumber
        self.num_cols = columnNumber
        
        self.grid = []
        for row in range(rowNumber):

            line = []
            for column in range(columnNumber):
                line.append(Cell(self, column, row, cellSize) if init_grid==None else Cell(self, column, row, cellSize, init_grid[row][column].state))

            self.grid.append(line)

        #memorize the cells that have been modified to avoid many switching of state during mouse motion.
        self.switched = []

        #bind click action
        self.bind("<Button-1>", self.handleMouseClick)  
        #bind moving while clicking
        self.bind("<B1-Motion>", self.handleMouseMotion)
        #bind release button action - clear the memory of midified cells.
        self.bind("<ButtonRelease-1>", lambda event: self.switched.clear())

        self.draw()



    def draw(self):
        for row in self.grid:
            for cell in row:
                cell.draw()

    def _eventCoords(self, event):
        row = int(event.y / self.cellSize)
        column = int(event.x / self.cellSize)
        return row, column

    def handleMouseClick(self, event):
        row, column = self._eventCoords(event)
        cell = self.grid[row][column]
        cell._switch()
        cell.draw()
        #add the cell to the list of cell switched during the click
        self.switched.append(cell)

    def handleMouseMotion(self, event):
        row, column = self._eventCoords(event)
        cell = self.grid[row%self.num_rows][column%self.num_cols]

        if cell not in self.switched:
            cell._switch()
            cell.draw()
            self.switched.append(cell)



app = Tk()

Grid = CellGrid(app, 50, 50, 20)
Grid.pack()

app.mainloop()


## Toggle Pedestrian Motion
Upon clicking the Left Mouse Button, the user can now toggle pedestrian movement towards the target

In [9]:
class Simulation(Canvas):
    def __init__(self, root, rowNumber, columnNumber, cellSize, init_grid, pedestrian, target, *args, **kwargs):
        Canvas.__init__(self, root, width = cellSize * columnNumber , height = cellSize * rowNumber, *args, **kwargs)
        
        self.grid = []
        self.row_size = rowNumber
        self.col_size = columnNumber
        self.pedestrian = pedestrian
        self.target = target
        #self.grid = init_grid
        for row in range(rowNumber):
            line = []
            for column in range(columnNumber):
                line.append(Cell(self, column, row, cellSize) if init_grid==None else Cell(self, column, row, cellSize, init_grid[row][column].state))

            self.grid.append(line)
        
        self.draw()
        
        self.bind("<Button-1>", self.handleMouseClick)
        
                
    def draw(self):
        for row in self.grid:
            for cell in row:
                cell.draw(cell.state)
                
    def handleMouseClick(self, event):
        self.grid, self.pedestrian = update(self.grid, self.pedestrian, self.target)
        self.draw()
    

##Accumulate Pedestrians and Target from the grid
pedestrians = []
for row in range(len(Grid.grid)):
    for col in range(len(Grid.grid[0])):
        if Grid.grid[row][col].state=='pedestrian':
            pedestrians.append([col+1,row+1])
        elif Grid.grid[row][col].state=='target':
            target = [col+1,row+1]

root = Tk()

application = Simulation(root, 50, 50, 20, Grid.grid, pedestrians, target)
application.pack()

root.mainloop()

    
    